In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model

In [2]:
pd.set_option("display.max_rows", None,
             "display.max_columns", None)

In [3]:
dataset = pd.read_csv("/Users/ecemdenizbabaoglan/yap470-proje/new_df_kucuk_v3.csv")

In [4]:
labels = dataset['Label']
data = dataset.loc[:, dataset.columns != 'Label']

In [5]:
labels.value_counts()

Benign                    837030
DDOS attack-HOIC           97811
Bot                        41017
FTP-BruteForce             27814
SSH-Bruteforce             27123
DDoS attacks-LOIC-HTTP     11123
DoS attacks-GoldenEye       6104
DoS attacks-Slowloris       1555
DDOS attack-LOIC-UDP         259
Brute Force -Web             104
Brute Force -XSS              42
SQL Injection                 18
Name: Label, dtype: int64

In [6]:
labels.loc[labels == 'Benign'].shape

(837030,)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size = 0.2, random_state = 12345, stratify = labels)

In [8]:
y_train.value_counts()

Benign                    669624
DDOS attack-HOIC           78249
Bot                        32814
FTP-BruteForce             22251
SSH-Bruteforce             21699
DDoS attacks-LOIC-HTTP      8898
DoS attacks-GoldenEye       4883
DoS attacks-Slowloris       1244
DDOS attack-LOIC-UDP         207
Brute Force -Web              83
Brute Force -XSS              34
SQL Injection                 14
Name: Label, dtype: int64

In [9]:
benign_train_data = x_train.loc[y_train == 'Benign']
attack_train_data = x_train.loc[y_train != 'Benign']
benign_test_data = x_test.loc[y_test == 'Benign']
attack_test_data = x_test.loc[y_test != 'Benign']

In [10]:
attack_train_data.shape

(170376, 82)

In [11]:
y_train.loc[y_train != 'Benign'] = 1
y_train.loc[y_train == 'Benign'] = 2
y_test.loc[y_test != 'Benign'] = 1
y_test.loc[y_test == 'Benign'] = 2

In [12]:
x_train = np.asarray(x_train).astype(np.float32)
x_test = np.asarray(x_test).astype(np.float32)

x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

ValueError: could not convert string to float: '15/02/2018 04:03:36'

In [ ]:
benign_train_data.shape

In [ ]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(32, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu")])

    self.decoder = tf.keras.Sequential([
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(140, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mae')

In [ ]:
history = autoencoder.fit(benign_train_data, benign_test_data, 
          epochs=20, 
          batch_size=512,
          validation_data=(x_test, x_test),
          shuffle=True)